In [1]:
# importing src directory
import sys
sys.path.append('..')
# experiment imports
import os
import math
import numpy as np
import random
from datetime import datetime as dt
from scipy.stats import truncnorm
import matplotlib.pyplot as plt
# project imports
from amm.amm import AMM, SimpleFeeAMM
from amm.fee import TriangleFee, PercentFee, NoFee
# data imports
from gbm.gbm import get_gbm_data, calibrate_gbm
from api_key.my_api_key import api_key

In [2]:

def sim1(n, pair, start_dt, end_dt, frequency, L0=1000000, spread=0.5):
    """
    simulate AMM market with data calibrated GBM for external oracles and trading agents
    n (int): number of simulations
    pair (str): asset pair for data (e.g. btc-eth)
    asset1_n (int): number of asset1 tokens
    asset2_n (int): number of asset2 tokens
    start_dt (str): start date for data (YYYY-MM-DD)
    end_dt (str): end date for data (YYYY-MM-DD)
    frequency (str): frequency of data (1h, 1d, 1w)
    L0 (int): number of initial LP tokens
    spread (float): spread for arbitrage agents (e.g. 0.5%)
    return list: list of dataframes for each simulation 
    """

    # # SIM STORAGE # #
    # create list to store dfs from each simulation of amms
    sim_amm_dfs= []
    sim_amms = []
    # parse asset1 and asset2, create USD denominated pairs
    asset1 = pair.split("-")[0] 
    asset2 = pair.split("-")[1]

    # # DATA & GBM CALIBRATION # #
    difference = dt.strptime(end_dt, '%Y-%m-%dT%H:%M:%SZ') - dt.strptime(start_dt, '%Y-%m-%dT%H:%M:%SZ')
    T_years = difference.days / 365.25  # using 365.25 to account for leap years
    marketDF =  get_gbm_data(pair, start_dt, end_dt, frequency, api_key) # get data for assets
    n_timesteps = len(marketDF) # number of timesteps in data
    new_cols = [f'gbm_price_{asset1}', f'gbm_price_{asset2}', # inventory of each asset
                f'{asset1}_inv', f'{asset2}_inv', 'L_inv', f'F{asset1}_inv', f'F{asset2}_inv', 
                'FL_inv', f'dt_{asset1}', f'dt_{asset2}', 'dt_L', f'dt_F{asset1}', f'dt_L']
    marketDF = marketDF.assign(**{col: None for col in new_cols})
    
    L0 = 1000000 # 1 mil LP tokens
    A0 = math.sqrt(L0**2 * marketDF[f'price_{asset1}'][0]/marketDF[f'price_{asset2}'][0]) # evenly distribute assets
    B0 = L0**2 / A0
    market_portfolio = {"A": A0, "B": B0, "L":L0} # initial portfolio 
    
    # gbm_assumption_test(np.log(1 + marketDF[f"price_{asset1}"].pct_change().dropna())) # test gbm assumptions
    # gbm_assumption_test(np.log(1 + marketDF[f"price_{asset2}"].pct_change().dropna())) # test gbm assumptions

    # # TIME SERIES SIMULATIONS # #
    
    print("HERE 1")

    # # TODO: ISSUE BELOW
    for simulation in range(n): # for each simulation create new set of amms & run new set of trades
        _,_,marketDF[f'gbm_price_{asset1}'] = calibrate_gbm(asset1, marketDF[f"price_{asset1}"], frequency, T_years, n_timesteps, "mle") # calibrate gbm for asset1 w/ MLE
        _,_,marketDF[f'gbm_price_{asset2}'] = calibrate_gbm(asset2, marketDF[f"price_{asset2}"], frequency, T_years, n_timesteps, "mle") # calibrate gbm for asset2 w/ MLE
        marketDF[f'{asset1}_inv'][0]= A0 # set initial asset1 inventory
        marketDF[f'{asset2}_inv'][0]= B0 # set initial asset2 inventory
        
        print("HERE 2")

        nofeeAMM = SimpleFeeAMM(fee_structure = NoFee(), initial_portfolio=market_portfolio)
        percentAMM = SimpleFeeAMM(fee_structure = PercentFee(0.01), initial_portfolio=market_portfolio)
        triAMM = SimpleFeeAMM(fee_structure = TriangleFee(0.003, 0.0001, -1), initial_portfolio=market_portfolio) 
        percentDF = marketDF.copy(deep=True)
        nofeeDF = marketDF.copy(deep=True)
        triDF = marketDF.copy(deep=True)
        amms = [(nofeeAMM, nofeeDF), (percentAMM, percentDF), (triAMM, triDF)] # store pairs of amm type & df for updating
        
# TODO: fix here ^
        # # SIMULATION # #
        for t in range(n_timesteps): # iterate over each timestep in crypto market data
            
            print("HERE 3")
            print(marketDF)

            # # ARBITRAGE AGENT # #
            
            for amm, df in amms: # update market data with amm data

                if marketDF[f'amm_{asset1}/{asset2}'][t] > (marketDF[f'gbm_{asset1}/{asset2}'][t] * (1+spread/100)): # rule-based arbitrage agents in the market
                    asset_out, asset_in, asset_in_n = asset1, asset2, random.choice(list(range(1, 50))) # modeling market efficiency
                if (marketDF[f'amm_{asset1}/{asset2}'][t] * 1.005) < marketDF[f'gbm_{asset1}/{asset2}'][t]:
                    asset_out, asset_in, asset_in_n = asset2, asset1, random.choice(list(range(1, 50)))
                else: continue

                print("HERE 4")

                succ, info = amm.trade_swap(asset_out, asset_in, asset_in_n) # call trade for each AMM
                new_row = {f'{asset1}_inv': amm.portfolio[asset1], f'{asset2}_inv': amm.portfolio[asset2], # add trade info to df
                           'LInv': amm.portfolio['L'], asset1: info['asset_delta'][asset1], 
                           f'{asset2}': info['asset_delta'][asset2], 'L': info['asset_delta']['L'], 
                        f'F{asset1}': amm.fees[asset1], f'F{asset2}': amm.fees[asset2], 'FL': amm.fees['L']}
                df.loc[t] = new_row # append new row to df
                # df.append(new_row, ignore_index=True)               
                
                print("HERE 5")                                                                            # TRYING TO FIX BUG !!!!

        for amm, df in amms:
            sim_amm_dfs.append(df)
            sim_amms.append(amm)
    return sim_amm_dfs, sim_amms # return list of dfs for each simulation

In [3]:
sim1(2, "btc-eth", '2023-02-01T00:00:00Z', '2024-02-03T00:00:00Z', "1d")

OSError: Cannot save file into a non-existent directory: 'data/crypto_data'

In [ ]:
# # NOTES FROM LAST MEETING:
# FOCUS MORE ON TESTING FEES THROUGH SIM

# # EXPERIMENTS TODO: # #
# [1] run for large simulations and evaluate over time - explore different time periods to test from (different market conditions and lengths of historical windows) and different frequencies (1h, 1d, 1w)
# [2] identify GBM paths that deplete pools (depletion of liquidity) and have both fall in value (impermanent loss) to show how fee accumulation compares ot general trend (law of large #s)
        # impermanent loss evaluation could allow for an expected value calculation for LP returns (expected value of fees vs. impermanent loss)
# [3] use stock data to see how compares
# [4] make sure to highlight how different fee AMMs (basically fees) are affected by different market conditions and therefore how fee accumulation is affected

# # UPDATES # #
# [1] *importing stock data to use instead of crypto (more in line with goal application and can properly use GBM to simulate)
# [2] considering train/test split for calibrating GBM and simulating trades source data (not overly urgent given not forecasting)
# [3] maybe also considering changing source data from vwap if stick with crypto data
        # multiple price streams for multiple external oracles